In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv()

In [ ]:
import pandas as pd

# Load your CSV dataset into a DataFrame
df = pd.read_csv('your_dataset.csv')

# Explore your data
print(df.head())  # Display the first few rows to understand the structure

# Preprocess your data
# For simplicity, let's assume you have a 'target_variable' column and a few feature columns
# Replace missing values, if any
df.fillna(0, inplace=True)  # Replace NaN values with 0; you might need a more sophisticated approach

# Select features and target variable
X = df[['feature1', 'feature2', ...]]  # Features
y = df['target_variable']  # Target variable

# Split data into training and testing sets
# For simplicity, we'll use a basic split without any randomization
train_size = int(0.8 * len(df))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build a simple decision tree model
def mean(values):
    return sum(values) / len(values)

def split_dataset(X, y, value, column):
    left_mask = X[column] <= value
    right_mask = X[column] > value
    return X[left_mask], X[right_mask], y[left_mask], y[right_mask]

def calculate_mae(y):
    return sum(abs(y - mean(y))) / len(y)

def decision_tree(X, y, max_depth=None):
    if max_depth == 0 or len(set(y)) == 1:
        return mean(y)

    best_mae = float('inf')
    best_value, best_column = None, None

    for column in X.columns:
        for value in set(X[column]):
            left_X, right_X, left_y, right_y = split_dataset(X, y, value, column)
            current_mae = calculate_mae(left_y) + calculate_mae(right_y)

            if current_mae < best_mae:
                best_mae = current_mae
                best_value, best_column = value, column

    if best_value is None or best_column is None:
        return mean(y)

    left_X, right_X, left_y, right_y = split_dataset(X, y, best_value, best_column)
    left_branch = decision_tree(left_X, left_y, max_depth - 1)
    right_branch = decision_tree(right_X, right_y, max_depth - 1)

    return {'value': best_value, 'column': best_column, 'left': left_branch, 'right': right_branch}

# Train the decision tree
max_depth = 3  # Adjust the maximum depth as needed
tree_model = decision_tree(X_train, y_train, max_depth)

# Make predictions
def predict(tree, row):
    if isinstance(tree, (int, float)):
        return tree

    if row[tree['column']] <= tree['value']:
        return predict(tree['left'], row)
    else:
        return predict(tree['right'], row)

predictions = [predict(tree_model, row) for _, row in X_test.iterrows()]

# Evaluate model performance
mae = calculate_mae(predictions)
print(f'Mean Absolute Error: {mae}')
